# Graph Neural Network Using Neo4J & PyTorch

### Environment

In [85]:
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import pandas as pd
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [86]:
from py2neo import Graph

In [87]:
graph = Graph("bolt://54.87.236.196:37389", auth=("neo4j", "throttle-retailers-jobs"))

#### EDA

In [88]:
query = """
MATCH (p:Person) RETURN count(*) as numNodes
"""
graph.run(query).to_data_frame()

,numNodes
0,34


## Graph Operations and Functions

Setting labels for node 1 and 34

In [89]:
query_instructor = """
MATCH (p:Person) WHERE p.id = 1 SET p.label = 0 
"""
query_president = """
MATCH (p:Person) WHERE p.id = 34 SET p.label = 1 
"""
graph.run(query_instructor)
graph.run(query_president)

Setting features of all the nodes using a dictionary

In [312]:
# aux function 
def stringToIntArray(word):
    return [int(i) for i in word]

def arrayToString(array):
    word = ""
    for i in range(0, array.size):
        word += str(array[i])
    return word

In [316]:
def set_features(graph, features):
    # features = dict{key:features}
    for key in features.keys():
        query = """
        MATCH (p:Person {id: $id_node}) SET p.features = $features
        """ 
        graph.run(query, {'id_node': key, 'features': arrayToString(features[key])})

In [314]:
dict_features = {}
num_nodes = 34
features = np.transpose(np.identity(num_nodes, dtype=int))
for i in range(1, num_nodes + 1):
    dict_features[i] = features[num_nodes-i]

In [317]:
set_features(graph, dict_features)

Either add self loops in the graph or take this into account when updating features of each node
# ¡¡¡¡¡!!!!! Ajustarlo después -> eliminar los self loops del gráfico?? modificar la funcion de update_features y añadir previamente los self loops?

In [310]:
def get_features(graph, id_node):
    query_own_features = """
    MATCH (p:Person {id: $id_node}) RETURN p.features as own_features
    """  
    own_features = graph.run(query_own_features, {"id_node": id_node}).to_data_frame().iloc[0]['own_features']
    return own_features

def get_neighbors_features(graph, id_node):
    query_neighbors_features = """
    MATCH (p:Person {id: $id_node})-[:KNOWS]-(neighbor:Person)
    RETURN collect(neighbor.features) as neighbors_features
    """
    neighbor_features = graph.run(query_neighbors_features, {"id_node": id_node}).to_data_frame().iloc[0]['neighbors_features']
    return neighbor_features

def update_features(graph, id_node, all_features, all_neighbor_features):
    own_features = stringToIntArray(all_features[id_node])
    neighbors_features = list(map(stringToIntArray, all_neighbor_features[id_node]))
    new_features = arrayToString(np.sum(neighbors_features, axis=0) + own_features)
    query_update_features = """
    MATCH (p:Person {id: $id_node}) SET p.features = $new_features
    """
    graph.run(query_update_features, {'id_node': id_node, 'new_features': new_features})

Obtain features for each node and obtain features of the neighbors of each node

In [318]:
def get_all_features(graph): 
    all_features = {}
    all_neighbor_features = {}
    for i in range(1, 34):
        i_features = get_features(graph, i)
        all_features[i] = i_features
        neighbor_features = get_neighbors_features(graph, i)
        all_neighbor_features[i] = neighbor_features
    return all_features, all_neighbor_features

In [311]:
query_own_features = """
    MATCH (p:Person {id: $id_node}) RETURN p.features as own_features
    """  
own_features = graph.run(query_own_features, {"id_node": 34}).to_data_frame()
own_features

,own_features
0,None


In [268]:
for i in range(1, 34):
    update_features(graph, 6, all_features, all_neighbor_features)

## Neural Network Creation

In [270]:
import torch
import torch.nn as nn

In [183]:
class GCNLayer(nn.Module):
    def __init__(self, input_features, output_features):
        super(GCNLayer, self).__init__()
        self.linear = Linear(input_features, output_features)
    def __forward__(self, graph, input):
        set_features(graph, input)
        all_features, all_neighbor_features = get_all_features(graph)
        

'0000000000000000000000000000010000'